In [ ]:
import json

from statistics import mean
from pprint import pprint

def get_percentage(num, den):
  if den == 0:
    return 0
  return round((num / den * 100), 2)

In [ ]:
with open("dataset_final.json") as jsonfile:
  data = json.load(jsonfile)

In [ ]:
def get_acc_rcl(lst, num_instances):
  ACC = get_percentage(
      lst.count(True),
      len(lst)
  )
  RCL = get_percentage(
      lst.count(True),
      num_instances
  )
  return ACC, RCL

# RQ1 - Tool Comparison


In [ ]:
num_positive = len(list(filter(lambda x: x['part2_aggregate']['polarity'] == "positive", data)))
num_neutral = len(list(filter(lambda x: x['part2_aggregate']['polarity'] == "neutral", data)))
num_negative = len(list(filter(lambda x: x['part2_aggregate']['polarity'] == "negative", data)))

for message in data:
  if "discussion_polarity" in message:
    if message["discussion_polarity"] == "positive":
      num_positive += 1
    elif message["discussion_polarity"] == "negative":
      num_negative += 1
    else:
      num_neutral += 1

results_per_class = {
    "positive": {
        "SentiStrength": [],
        "SentiStrengthSE": [],
        "DEVA": [],
        "SentiCR": [],
        "Senti4SD": []
    },
    "neutral": {
        "SentiStrength": [],
        "SentiStrengthSE": [],
        "DEVA": [],
        "SentiCR": [],
        "Senti4SD": []
    },
    "negative": {
        "SentiStrength": [],
        "SentiStrengthSE": [],
        "DEVA": [],
        "SentiCR": [],
        "Senti4SD": []
    }
}

for message in data:
  real_polarity = message['part2_aggregate']['polarity']
  if real_polarity == "undefined":
    real_polarity = message["discussion_polarity"]
  for tool, polarity in message['tools'].items():
    results_per_class[polarity][tool].append(polarity == real_polarity)

SentiCR_data = []

for message in data:
  real_polarity = message['part2_aggregate']['polarity']
  if real_polarity == "undefined":
    real_polarity = message["discussion_polarity"]
  senticr_polarity = message['tools']['SentiCR']

  if senticr_polarity == "negative":
    if real_polarity == "negative":
      SentiCR_data.append("TP")
    else:
      SentiCR_data.append("FP")
  else:
    if real_polarity == "negative":
      SentiCR_data.append("FN")
    else:
      SentiCR_data.append("TN")

In [ ]:
print("----- POSITIVE: -----")

print("SentiStrength:   ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['positive']['SentiStrength'], num_positive)))
print("SentiStrengthSE: ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['positive']['SentiStrengthSE'], num_positive)))
print("DEVA:            ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['positive']['DEVA'], num_positive)))
print("Senti4SD:        ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['positive']['Senti4SD'], num_positive)))

print()
print("----- NEUTRAL: -----")

print("SentiStrength:   ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['neutral']['SentiStrength'], num_neutral)))
print("SentiStrengthSE: ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['neutral']['SentiStrengthSE'], num_neutral)))
print("DEVA:            ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['neutral']['DEVA'], num_neutral)))
print("Senti4SD:        ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['neutral']['Senti4SD'], num_neutral)))

print()
print("----- NEGATIVE: -----")

print("SentiStrength:   ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['negative']['SentiStrength'], num_negative)))
print("SentiStrengthSE: ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['negative']['SentiStrengthSE'], num_negative)))
print("DEVA:            ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['negative']['DEVA'], num_negative)))
print("Senti4SD:        ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['negative']['Senti4SD'], num_negative)))
print("SentiCR          ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['negative']['SentiCR'], num_negative)))

print()
print("----- GENERAL -----:")

for tool in ["SentiStrength", "SentiStrengthSE", "Senti4SD", "DEVA"]:
  ACC_1, RCL_1 = get_acc_rcl(results_per_class['positive'][tool], num_positive)
  ACC_2, RCL_2 = get_acc_rcl(results_per_class['neutral'][tool], num_neutral)
  ACC_3, RCL_3 = get_acc_rcl(results_per_class['negative'][tool], num_negative)
  print("{}   ACC {}% RCL {}%".format(
    (tool + ":").ljust(16, ' '),
    round(mean([ACC_1, ACC_2, ACC_3]), 2),
    round(mean([RCL_1, RCL_2, RCL_3]), 2)
  ))
print("SentiCR:           ACC {}% RCL {}%".format(
    get_percentage(
        SentiCR_data.count("TP"),
        SentiCR_data.count("TP") + SentiCR_data.count("FP")
    ),
    get_percentage(
        SentiCR_data.count("TP"),
        SentiCR_data.count("TP") + SentiCR_data.count("FN")
    )
))


----- POSITIVE: -----
SentiStrength:   ACC 55.32% RCL 78.89%
SentiStrengthSE: ACC 79.23% RCL 62.96%
DEVA:            ACC 66.49% RCL 72.36%
Senti4SD:        ACC 56.47% RCL 71.21%

----- NEUTRAL: -----
SentiStrength:   ACC 80.53% RCL 36.52%
SentiStrengthSE: ACC 79.32% RCL 55.37%
DEVA:            ACC 79.87% RCL 56.8%
Senti4SD:        ACC 65.81% RCL 63.84%

----- NEGATIVE: -----
SentiStrength:   ACC 45.06% RCL 69.68%
SentiStrengthSE: ACC 43.56% RCL 79.86%
DEVA:            ACC 48.41% RCL 70.37%
Senti4SD:        ACC 57.32% RCL 42.59%
SentiCR          ACC 42.79% RCL 39.81%

----- GENERAL -----:
SentiStrength:     ACC 60.3% RCL 61.7%
SentiStrengthSE:   ACC 67.37% RCL 66.06%
Senti4SD:          ACC 59.87% RCL 59.21%
DEVA:              ACC 64.92% RCL 66.51%
SentiCR:           ACC 42.79% RCL 39.81%


# RQ1 - Tool comparison Per Agreement Level

In [ ]:
agreement_map = {
    "all": "all",
    "comp_only": "partial",
    "neuro_and_comp": "partial",
    "undefined": "none"
}

agreement_count = {
    "all": {
      "positive": 0,
      "neutral": 0,
      "negative": 0
  },
    "partial": {
      "positive": 0,
      "neutral": 0,
      "negative": 0
  },
    "none": {
      "positive": 0,
      "neutral": 0,
      "negative": 0
  }
}

for message in data:
  agreement_type = message['part2_aggregate']['agreement_type']
  polarity = message['part2_aggregate']['polarity']
  if agreement_type == "undefined":
    polarity = message["discussion_polarity"]
    agreement_count["none"][polarity] += 1
  elif agreement_type == "all":
    agreement_count["all"][polarity] += 1
  else:
    agreement_count["partial"][polarity] += 1

results_per_agreement_type = {}
for agreement_type in ['all', "partial", "none"]:
  results_per_agreement_type[agreement_type] = {
    "positive": {
        "SentiStrength": [],
        "SentiStrengthSE": [],
        "DEVA": [],
        "SentiCR": [],
        "Senti4SD": []
    },
    "neutral": {
        "SentiStrength": [],
        "SentiStrengthSE": [],
        "DEVA": [],
        "SentiCR": [],
        "Senti4SD": []
    },
    "negative": {
        "SentiStrength": [],
        "SentiStrengthSE": [],
        "DEVA": [],
        "SentiCR": [],
        "Senti4SD": []
    }
}

for message in data:
  agreement_type = message['part2_aggregate']['agreement_type']
  real_polarity = message['part2_aggregate']['polarity']
  if real_polarity == "undefined":
    real_polarity = message["discussion_polarity"]
  for tool, polarity in message['tools'].items():
    results_per_agreement_type[agreement_map[agreement_type]][polarity][tool].append(polarity == real_polarity)

In [ ]:
print(agreement_count)

for agreement_type in ['all', "partial", "none"]:
  num_positive = agreement_count[agreement_type]['positive']
  num_negative = agreement_count[agreement_type]['negative']
  num_neutral = agreement_count[agreement_type]['neutral']


  print()
  print("----------------- AGREEMENT TYPE {}: ---------------------------".format(agreement_type))
  print()
  print("----- POSITIVE: -----")

  print("SentiStrength:   ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['positive']['SentiStrength'], num_positive)))
  print("SentiStrengthSE: ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['positive']['SentiStrengthSE'], num_positive)))
  print("DEVA:            ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['positive']['DEVA'], num_positive)))
  print("Senti4SD:        ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['positive']['Senti4SD'], num_positive)))

  print()
  print("----- NEUTRAL: -----")

  print("SentiStrength:   ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['neutral']['SentiStrength'], num_neutral)))
  print("SentiStrengthSE: ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['neutral']['SentiStrengthSE'], num_neutral)))
  print("DEVA:            ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['neutral']['DEVA'], num_neutral)))
  print("Senti4SD:        ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['neutral']['Senti4SD'], num_neutral)))

  print()
  print("----- NEGATIVE: -----")

  print("SentiStrength:   ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['negative']['SentiStrength'], num_negative)))
  print("SentiStrengthSE: ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['negative']['SentiStrengthSE'], num_negative)))
  print("DEVA:            ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['negative']['DEVA'], num_negative)))
  print("Senti4SD:        ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['negative']['Senti4SD'], num_negative)))
  print("SentiCR          ACC {}% RCL {}%".format(*get_acc_rcl(results_per_agreement_type[agreement_type]['negative']['SentiCR'], num_negative)))

  print()
  print("----- GENERAL -----:")

  for tool in ["SentiStrength", "SentiStrengthSE", "Senti4SD", "DEVA"]:
    ACC_1, RCL_1 = get_acc_rcl(results_per_agreement_type[agreement_type]['positive'][tool], num_positive)
    ACC_2, RCL_2 = get_acc_rcl(results_per_agreement_type[agreement_type]['neutral'][tool], num_neutral)
    ACC_3, RCL_3 = get_acc_rcl(results_per_agreement_type[agreement_type]['negative'][tool], num_negative)
    print("{}   ACC {}% RCL {}%".format(
      (tool + ":").ljust(16, ' '),
      round(mean([ACC_1, ACC_2, ACC_3]), 2),
      round(mean([RCL_1, RCL_2, RCL_3]), 2)
    ))

{'all': {'positive': 331, 'neutral': 489, 'negative': 222}, 'partial': {'positive': 177, 'neutral': 318, 'negative': 205}, 'none': {'positive': 13, 'neutral': 31, 'negative': 5}}

----------------- AGREEMENT TYPE all: ---------------------------

----- POSITIVE: -----
SentiStrength:   ACC 64.29% RCL 87.01%
SentiStrengthSE: ACC 87.54% RCL 74.32%
DEVA:            ACC 78.19% RCL 83.38%
Senti4SD:        ACC 64.1% RCL 80.36%

----- NEUTRAL: -----
SentiStrength:   ACC 90.91% RCL 44.99%
SentiStrengthSE: ACC 92.33% RCL 64.01%
DEVA:            ACC 92.09% RCL 66.67%
Senti4SD:        ACC 73.48% RCL 69.12%

----- NEGATIVE: -----
SentiStrength:   ACC 50.57% RCL 80.18%
SentiStrengthSE: ACC 48.1% RCL 91.44%
DEVA:            ACC 56.12% RCL 84.68%
Senti4SD:        ACC 69.46% RCL 52.25%
SentiCR          ACC 47.59% RCL 40.09%

----- GENERAL -----:
SentiStrength:     ACC 68.59% RCL 70.73%
SentiStrengthSE:   ACC 75.99% RCL 76.59%
Senti4SD:          ACC 69.01% RCL 67.24%
DEVA:              ACC 75.47% RCL 78

# RQ2 - Tool Comparison


In [ ]:
num_positive = len(list(filter(lambda x: x['part1_aggregate']['polarity'] == "positive", data)))
num_neutral = len(list(filter(lambda x: x['part1_aggregate']['polarity'] == "neutral", data)))
num_negative = len(list(filter(lambda x: x['part1_aggregate']['polarity'] == "negative", data)))

results_per_class = {
    "positive": {
        "SentiStrength": [],
        "SentiStrengthSE": [],
        "DEVA": [],
        "SentiCR": [],
        "Senti4SD": []
    },
    "neutral": {
        "SentiStrength": [],
        "SentiStrengthSE": [],
        "DEVA": [],
        "SentiCR": [],
        "Senti4SD": []
    },
    "negative": {
        "SentiStrength": [],
        "SentiStrengthSE": [],
        "DEVA": [],
        "SentiCR": [],
        "Senti4SD": []
    }
}

for message in data:
  if "discussion_polarity" in message:
    continue
  real_polarity = message['part1_aggregate']['polarity']
  for tool, polarity in message['tools'].items():
    results_per_class[polarity][tool].append(polarity == real_polarity)

SentiCR_data = []

for message in data:
  if "discussion_polarity" in message:
    continue
  real_polarity = message['part1_aggregate']['polarity']
  senticr_polarity = message['tools']['SentiCR']

  if senticr_polarity == "negative":
    if real_polarity == "negative":
      SentiCR_data.append("TP")
    else:
      SentiCR_data.append("FP")
  else:
    if real_polarity == "negative":
      SentiCR_data.append("FN")
    else:
      SentiCR_data.append("TN")


In [ ]:
print("----- POSITIVE: -----")

print("SentiStrength:   ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['positive']['SentiStrength'], num_positive)))
print("SentiStrengthSE: ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['positive']['SentiStrengthSE'], num_positive)))
print("DEVA:            ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['positive']['DEVA'], num_positive)))
print("Senti4SD:        ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['positive']['Senti4SD'], num_positive)))

print()
print("----- NEUTRAL: -----")

print("SentiStrength:   ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['neutral']['SentiStrength'], num_neutral)))
print("SentiStrengthSE: ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['neutral']['SentiStrengthSE'], num_neutral)))
print("DEVA:            ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['neutral']['DEVA'], num_neutral)))
print("Senti4SD:        ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['neutral']['Senti4SD'], num_neutral)))

print()
print("----- NEGATIVE: -----")

print("SentiStrength:   ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['negative']['SentiStrength'], num_negative)))
print("SentiStrengthSE: ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['negative']['SentiStrengthSE'], num_negative)))
print("DEVA:            ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['negative']['DEVA'], num_negative)))
print("Senti4SD:        ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['negative']['Senti4SD'], num_negative)))
print("SentiCR          ACC {}% RCL {}%".format(*get_acc_rcl(results_per_class['negative']['SentiCR'], num_negative)))

print()
print("----- GENERAL -----:")

for tool in ["SentiStrength", "SentiStrengthSE", "Senti4SD", "DEVA"]:
  ACC_1, RCL_1 = get_acc_rcl(results_per_class['positive'][tool], num_positive)
  ACC_2, RCL_2 = get_acc_rcl(results_per_class['neutral'][tool], num_neutral)
  ACC_3, RCL_3 = get_acc_rcl(results_per_class['negative'][tool], num_negative)
  print("{}   ACC {}% RCL {}%".format(
    (tool + ":").ljust(16, ' '),
    round(mean([ACC_1, ACC_2, ACC_3]), 2),
    round(mean([RCL_1, RCL_2, RCL_3]), 2)
  ))
print("SentiCR:           ACC {}% RCL {}%".format(
    get_percentage(
        SentiCR_data.count("TP"),
        SentiCR_data.count("TP") + SentiCR_data.count("FP")
    ),
    get_percentage(
        SentiCR_data.count("TP"),
        SentiCR_data.count("TP") + SentiCR_data.count("FN")
    )
))


----- POSITIVE: -----
SentiStrength:   ACC 56.01% RCL 79.41%
SentiStrengthSE: ACC 80.45% RCL 63.56%
DEVA:            ACC 67.82% RCL 73.47%
Senti4SD:        ACC 57.64% RCL 72.48%

----- NEUTRAL: -----
SentiStrength:   ACC 79.52% RCL 36.6%
SentiStrengthSE: ACC 78.82% RCL 55.57%
DEVA:            ACC 79.29% RCL 57.16%
Senti4SD:        ACC 65.11% RCL 63.28%

----- NEGATIVE: -----
SentiStrength:   ACC 44.77% RCL 67.99%
SentiStrengthSE: ACC 44.59% RCL 79.91%
DEVA:            ACC 49.83% RCL 70.56%
Senti4SD:        ACC 58.47% RCL 42.76%
SentiCR          ACC 43.91% RCL 40.42%

----- GENERAL -----:
SentiStrength:     ACC 60.1% RCL 61.33%
SentiStrengthSE:   ACC 67.95% RCL 66.35%
Senti4SD:          ACC 60.41% RCL 59.51%
DEVA:              ACC 65.65% RCL 67.06%
SentiCR:           ACC 43.91% RCL 40.8%
